<span style = "font-family: Verdana; font-size: 20px">

#### **Preprocessing**
</span>

<span style = "font-family: Verdana; font-size: 20px">

##### **Các bước thực hiện và ý tưởng**

Dựa trên đặc tính của bộ dữ liệu và mục tiêu dự án, các bước tiền xử lý dữ liệu dự kiến sẽ được thực hiện như sau:
- Xử lý các vấn đề tìm được trong cả 3 file dữ liệu
- Kết hợp dữ liệu từ `pokemon.csv` và `pokemonfullstats.csv` để tạo thành một bảng dữ liệu đầy đủ về đặc tính của các Pokemon
- Cuối cùng, kết hợp dữ liệu từ bảng đặc tính Pokemon với bảng trận đấu `combats.csv` để tạo thành bảng dữ liệu cuối cùng dùng để huấn luyện mô hình dự đoán kết quả trận đấu giữa hai Pokemon
</span>

In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [96]:
pokemon = pd.read_csv('data/pokemon.csv')
combats = pd.read_csv('data/combats.csv')
pokemonfullstats = pd.read_csv('data/pokemonfullstats.csv')

In [97]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        799 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


In [98]:
pokemonfullstats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179 entries, 0 to 1178
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DexNumber           1179 non-null   int64  
 1   Name                1179 non-null   object 
 2   Type                1179 non-null   object 
 3   Abilities           1179 non-null   object 
 4   HiddenAbility       1179 non-null   object 
 5   Generation          1179 non-null   object 
 6   Hp                  1179 non-null   int64  
 7   Attack              1179 non-null   int64  
 8   Defense             1179 non-null   int64  
 9   SpecialAttack       1179 non-null   int64  
 10  SpecialDefense      1179 non-null   int64  
 11  Speed               1179 non-null   int64  
 12  TotalStats          1179 non-null   int64  
 13  Weight              1179 non-null   float64
 14  Height              1179 non-null   float64
 15  GenderProbM         1179 non-null   object 
 16  Catego

In [99]:
combats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   First_pokemon   50000 non-null  int64
 1   Second_pokemon  50000 non-null  int64
 2   Winner          50000 non-null  int64
dtypes: int64(3)
memory usage: 1.1 MB


<span style = "font-family: Verdana; font-size: 20px">

##### **1. Missing Values**
</span>

In [100]:
na_pokemon = pokemon.isna().sum()
na_pokemonfs = pokemonfullstats.isna().sum()
na_combats = combats.isna().sum()
print(f'Missing value in pokemon: {sum(na_pokemon[na_pokemon > 0])}')
print(f'Missing value in pokemonfullstats: {sum(na_pokemonfs[na_pokemonfs > 0])}')
print(f'Missing value in combats: {sum(na_combats[na_combats > 0])}')

Missing value in pokemon: 387
Missing value in pokemonfullstats: 0
Missing value in combats: 0


<span style = "font-family: Verdana; font-size: 20px">

- Dựa vào hàm .info() ở trên ta thấy `pokemon.csv` có 1 missing value ở cột `Name` và rất nhiều missing values ở cột `Type 2` do nhiều Pokemon chỉ có một thuộc tính chính
</span>

<span style = "font-family: Verdana; font-size: 20px">

Ở cột Type 2, missing value có thể được giải quyết đơn giản bằng cách thêm giá trị 'None' để biểu thị cho việc Pokemon đó chỉ có một thuộc tính chính
</span>

In [101]:
pokemon.fillna({'Type 2': 'None'}, inplace=True)

<span style = "font-family: Verdana; font-size: 20px">

Còn ở cột `Name`, vì tên của các Pokemon thường sẽ được xếp theo thứ tự tiến hoá nên hãy xem qua các dòng xung quanh của missing value này
</span>

In [102]:
missing_point = pokemon.index[pokemon['Name'].isna()][0]
pokemon.loc[missing_point - 3:missing_point + 3, ['Name', 'Type 1', 'Type 2']]

,Name,Type 1,Type 2
59,Psyduck,Water,None
60,Golduck,Water,None
61,Mankey,Fighting,None
62,NaN,Fighting,None
63,Growlithe,Fire,None
64,Arcanine,Fire,None
65,Poliwag,Water,None


<span style = "font-family: Verdana; font-size: 20px">

Như vậy giá trị bị thiếu nằm giữa hai Pokemon có tên 'Mankey' và 'Growlithe', có thể suy luận giá trị từ các web tổng hợp thông tin pokemon, hoặc xem từ `pokemonfullstats.csv` vì dataset này không có giá trị thiếu trên cột `Name`

In [103]:
target_point = pokemonfullstats.index[pokemonfullstats['Name'] == 'Mankey'][0]
pokemonfullstats.loc[target_point - 2:target_point + 4, ['Name', 'Type']]

,Name,Type
65,Psyduck,['Water']
66,Golduck,['Water']
67,Mankey,['Fighting']
68,Primeape,['Fighting']
69,Growlithe,['Fire']
70,Hisuian Growlithe,"['Fire', 'Rock']"
71,Arcanine,['Fire']


<span style = "font-family: Verdana; font-size: 20px">

Có thể thấy giá trị bị thiếu chính là 'Primeape'
</span>

In [104]:
pokemon.loc[62, "Name"] = 'Primeape'

<span style = "font-family: Verdana; font-size: 20px">

Kiểm tra lại để chắc chắc không còn missing values nào trong toàn bộ dataset
</span>

In [105]:
na_pokemon = pokemon.isna().sum()
na_pokemonfs = pokemonfullstats.isna().sum()
na_combats = combats.isna().sum()
print(f'Missing value in pokemon: {sum(na_pokemon[na_pokemon > 0])}')
print(f'Missing value in pokemonfullstats: {sum(na_pokemonfs[na_pokemonfs > 0])}')
print(f'Missing value in combats: {sum(na_combats[na_combats > 0])}')

Missing value in pokemon: 0
Missing value in pokemonfullstats: 0
Missing value in combats: 0


<span style = "font-family: Verdana; font-size: 20px">

##### **2. Duplicating Values**
</span>

In [106]:
dup_pokemon = pokemon.duplicated().sum()
dup_pokemonfs = pokemonfullstats.duplicated().sum()
dup_combats = combats.duplicated().sum()
print(f'Duplicated value in pokemon: {sum(dup_pokemon[dup_pokemon > 0])}')
print(f'Duplicated value in pokemonfullstats: {sum(dup_pokemonfs[dup_pokemonfs > 0])}')
print(f'Duplicated value in combats: {sum(dup_combats[dup_combats > 0])}')

Duplicated value in pokemon: 0
Duplicated value in pokemonfullstats: 0
Duplicated value in combats: 1952


<span style = "font-family: Verdana; font-size: 20px">

Với dữ liệu lặp, cách xử lý đơn giản nhất là loại bỏ các dòng bị lặp đi
</span>

In [107]:
dup_combats_indices = combats[combats.duplicated()].index
combats = combats.drop(dup_combats_indices).reset_index(drop=True)

<span style = "font-family: Verdana; font-size: 20px">

Kiểm tra lại để chắc chắc không còn dữ liệu lặp nào trong toàn bộ dataset
</span>

In [108]:
dup_pokemon = pokemon.duplicated().sum()
dup_pokemonfs = pokemonfullstats.duplicated().sum()
dup_combats = combats.duplicated().sum()
print(f'Duplicated value in pokemon: {sum(dup_pokemon[dup_pokemon > 0])}')
print(f'Duplicated value in pokemonfullstats: {sum(dup_pokemonfs[dup_pokemonfs > 0])}')
print(f'Duplicated value in combats: {sum(dup_combats[dup_combats > 0])}')

Duplicated value in pokemon: 0
Duplicated value in pokemonfullstats: 0
Duplicated value in combats: 0


<span style = "font-family: Verdana; font-size: 20px">

##### **3. Các vấn đề đặc thù của bộ dữ liệu**
</span>

In [109]:
pd.options.display.max_columns = None

In [110]:
pokemon.head(10)

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,None,39,52,43,60,50,65,1,False
5,6,Charmeleon,Fire,None,58,64,58,80,65,80,1,False
6,7,Charizard,Fire,Flying,78,84,78,109,85,100,1,False
7,8,Mega Charizard X,Fire,Dragon,78,130,111,130,85,100,1,False
8,9,Mega Charizard Y,Fire,Flying,78,104,78,159,115,100,1,False
9,10,Squirtle,Water,None,44,48,65,50,64,43,1,False


In [111]:
pokemonfullstats.head(10)

,DexNumber,Name,Type,Abilities,HiddenAbility,Generation,Hp,Attack,Defense,SpecialAttack,SpecialDefense,Speed,TotalStats,Weight,Height,GenderProbM,Category,CatchRate,EggCycles,EggGroup,LevelingRate,BaseFriendship,IsLegendary,IsMythical,IsUltraBeast,HasMega,EvoStage,TotalEvoStages,PreevoName,DamageFromNormal,DamageFromFighting,DamageFromFlying,DamageFromPoison,DamageFromGround,DamageFromRock,DamageFromBug,DamageFromGhost,DamageFromSteel,DamageFromFire,DamageFromWater,DamageFromGrass,DamageFromElectric,DamageFromPsychic,DamageFromIce,DamageFromDragon,DamageFromDark,DamageFromFairy
0,1,Bulbasaur,"['Grass', 'Poison']",['Overgrow'],['Chlorophyll'],I,45,49,49,65,65,45,318,6.9,0.7,0.875,Seed Pokémon,45,20,['Monster' 'Grass'],Medium Slow\n,70,0,0,0,0,1,3,No Preevolution,1.0,0.5,2.0,1.0,1.0,1.0,1.00,1.0,1.0,2.0,0.5,0.25,0.5,2.0,2.0,1.0,1.0,0.5
1,2,Ivysaur,"['Grass', 'Poison']",['Overgrow'],['Chlorophyll'],I,60,62,63,80,80,60,405,13.0,1.0,0.875,Seed Pokémon,45,20,['Monster' 'Grass'],Medium Slow\n,70,0,0,0,0,2,3,Bulbasaur,1.0,0.5,2.0,1.0,1.0,1.0,1.00,1.0,1.0,2.0,0.5,0.25,0.5,2.0,2.0,1.0,1.0,0.5
2,3,Venusaur,"['Grass', 'Poison']",['Overgrow'],['Chlorophyll'],I,80,82,83,100,100,80,625,100.0,2.0,0.875,Seed Pokémon,45,20,['Monster' 'Grass'],Medium Slow\n,70,0,0,0,1,3,3,Ivysaur,1.0,0.5,2.0,1.0,1.0,1.0,1.00,1.0,1.0,2.0,0.5,0.25,0.5,2.0,2.0,1.0,1.0,0.5
3,4,Charmander,['Fire'],['Blaze'],['Solar Power'],I,39,52,43,60,50,65,309,8.5,0.6,0.875,Lizard Pokémon,45,20,['Monster' 'Dragon'],Medium Slow\n,70,0,0,0,0,1,3,No Preevolution,1.0,1.0,1.0,1.0,2.0,2.0,0.50,1.0,0.5,0.5,2.0,0.50,1.0,1.0,0.5,1.0,1.0,0.5
4,5,Charmeleon,['Fire'],['Blaze'],['Solar Power'],I,58,64,58,80,65,80,405,19.0,1.1,0.875,Flame Pokémon,45,20,['Monster' 'Dragon'],Medium Slow\n,70,0,0,0,0,2,3,Charmander,1.0,1.0,1.0,1.0,2.0,2.0,0.50,1.0,0.5,0.5,2.0,0.50,1.0,1.0,0.5,1.0,1.0,0.5
5,6,Charizard,"['Fire', 'Flying']",['Blaze'],['Solar Power'],I,78,84,78,109,85,100,634,90.5,1.7,0.875,Flame Pokémon,45,20,['Monster' 'Dragon'],Medium Slow\n,70,0,0,0,1,3,3,Charmeleon,1.0,0.5,1.0,1.0,0.0,4.0,0.25,1.0,0.5,0.5,2.0,0.25,2.0,1.0,1.0,1.0,1.0,0.5
6,7,Squirtle,['Water'],['Torrent'],['Rain Dish'],I,44,48,65,50,64,43,314,9.0,0.5,0.875,Tiny Turtle Pokémon,45,20,['Monster' 'Water 1'],Medium Slow\n,70,0,0,0,0,1,3,No Preevolution,1.0,1.0,1.0,1.0,1.0,1.0,1.00,1.0,0.5,0.5,0.5,2.00,2.0,1.0,0.5,1.0,1.0,1.0
7,8,Wartortle,['Water'],['Torrent'],['Rain Dish'],I,59,63,80,65,80,58,405,22.5,1.0,0.875,Turtle Pokémon,45,20,['Monster' 'Water 1'],Medium Slow\n,70,0,0,0,0,2,3,Squirtle,1.0,1.0,1.0,1.0,1.0,1.0,1.00,1.0,0.5,0.5,0.5,2.00,2.0,1.0,0.5,1.0,1.0,1.0
8,9,Blastoise,['Water'],['Torrent'],['Rain Dish'],I,79,83,100,85,105,78,630,85.5,1.6,0.875,Shellfish Pokémon,45,20,['Monster' 'Water 1'],Medium Slow\n,70,0,0,0,1,3,3,Wartortle,1.0,1.0,1.0,1.0,1.0,1.0,1.00,1.0,0.5,0.5,0.5,2.00,2.0,1.0,0.5,1.0,1.0,1.0
9,10,Caterpie,['Bug'],['Shield Dust'],['Run Away'],I,45,30,35,20,20,45,195,2.9,0.3,0.5,Worm Pokémon,255,15,['Bug'],Medium Fast\n,70,0,0,0,0,1,3,No Preevolution,1.0,0.5,2.0,1.0,0.5,2.0,1.00,1.0,1.0,2.0,1.0,0.50,1.0,1.0,1.0,1.0,1.0,1.0


<span style = "font-family: Verdana; font-size: 20px">

Để trực quan hoá một cách hiệu quả bộ dữ liệu về Pokemon, trước hết cần kết hợp thông tin từ cả hai bộ dữ liệu `pokemon.csv` và `pokemonfullstats.csv` để có được bảng dữ liệu đầy đủ về đặc tính của các Pokemon
Nhưng có một số vấn đề cần phải giải quyết trước khi thực hiện việc này:
- Dễ thấy cách đánh số ID của các Pokemon trong hai bộ dữ liệu không đồng nhất, ví dụ như trong `pokemon.csv`, Charmander có ID ở cột `#` là 5, nhưng trong `pokemonfullstats.csv`, lại có DexNumber là 4, và sự chênh lệch này sẽ lặp lại đến cuối bộ dữ liệu
</span>

In [112]:
pd.concat([pokemon[['#', 'Name']].head(10),pokemonfullstats[['DexNumber', 'Name']].head(10)], axis=1)

,#,Name,DexNumber,Name
0,1,Bulbasaur,1,Bulbasaur
1,2,Ivysaur,2,Ivysaur
2,3,Venusaur,3,Venusaur
3,4,Mega Venusaur,4,Charmander
4,5,Charmander,5,Charmeleon
5,6,Charmeleon,6,Charizard
6,7,Charizard,7,Squirtle
7,8,Mega Charizard X,8,Wartortle
8,9,Mega Charizard Y,9,Blastoise
9,10,Squirtle,10,Caterpie


<span style = "font-family: Verdana; font-size: 20px">

Nguyên nhân gây ra sự chênh lệch này là do trong `pokemonfullstats.csv`, dữ liệu được lấy từ [serebii.net](https://serebii.net/pokemon) và [bulbapedia](https://bulbapedia.bulbagarden.net/), và theo quy ước của các trang web này, các Pokemon tiến hoá ở bậc Mega sẽ nằm cùng một trang với Pokemon gốc của nó và có cùng số Pokedex, rất có thể thay đổi này được thực hiện sau khi bộ dữ liệu `pokemon.csv` được thu thập dẫn đến sự chênh lệch này

Tuy nhiên bộ dữ liệu `pokemon.csv` có cùng nguồn gốc với `combats.csv`, nên để đảm bảo tính nhất quán của dữ liệu, chúng em vẫn sẽ sử dụng cách đánh số ID từ `pokemon.csv` làm chuẩn, và `merge` hai bảng dữ liệu dựa trên cột `Name` thay vì cột `#` và `DexNumber` như thông thường, điều này đồng nghĩa cột `DexNumber` trong `pokemonfullstats.csv` sẽ không còn giá trị sử dụng
</span>

In [113]:
pokemonfullstats.drop(columns=['DexNumber'], inplace=True)

<span style = "font-family: Verdana; font-size: 20px">

- Ở cột `LevelingRate` trong `pokemonfullstats.csv`, có một ký tự xuống dòng thừa ở cuối mỗi giá trị, cần loại bỏ ký tự này để tránh lỗi khi xử lý dữ liệu sau này
</span>

In [114]:
pokemonfullstats['LevelingRate'] = pokemonfullstats['LevelingRate'].str.rstrip('\n')

<span style = "font-family: Verdana; font-size: 20px">

- Tiếp đó, trong cả 2 bộ dữ liệu cũng có một số cột có nội dung tương tự nhau, ví dụ như các chỉ số chiến đấu (HP, Attack, Defense, Special Attack, Special Defense, Speed), để tránh việc dư thừa thông tin khi kết hợp hai bảng dữ liệu, chúng em sẽ giữ lại các cột từ `pokemonfullstats.csv` vì bộ dữ liệu này có nhiều thông tin chi tiết hơn về các đặc điểm của Pokemon và loại bỏ các cột tương ứng từ `pokemon.csv`
</span>

In [115]:
pokemon.drop(columns=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'], inplace=True)

In [116]:
merged_pokemon = pd.merge(pokemon, pokemonfullstats, left_on='Name', right_on='Name', how='left')
merged_pokemon.sort_values(by=['#'], inplace=True)
merged_pokemon.rename(columns={'#': 'ID'}, inplace=True)

<span style = "font-family: Verdana; font-size: 20px">

##### **3.1. Missing Values**
</span>

In [117]:
merged_pokemon.isna().sum()

ID                     0
Name                   0
Type 1                 0
Type 2                 0
Type                  81
Abilities             81
HiddenAbility         81
Generation            81
Hp                    81
Attack                81
Defense               81
SpecialAttack         81
SpecialDefense        81
Speed                 81
TotalStats            81
Weight                81
Height                81
GenderProbM           81
Category              81
CatchRate             81
EggCycles             81
EggGroup              81
LevelingRate          81
BaseFriendship        81
IsLegendary           81
IsMythical            81
IsUltraBeast          81
HasMega               81
EvoStage              81
TotalEvoStages        81
PreevoName            81
DamageFromNormal      81
DamageFromFighting    81
DamageFromFlying      81
DamageFromPoison      81
DamageFromGround      81
DamageFromRock        81
DamageFromBug         81
DamageFromGhost       81
DamageFromSteel       81


<span style = "font-family: Verdana; font-size: 20px">

Sau khi kết hợp hai bảng dữ liệu, sẽ tạo ra kha khá missing values, nhưng điều này có thể dự đoán trước vì như đã đề cập ở trên, dữ liệu về các Pokemon sau khi tiến hoá bậc Mega không được thu thập, nhưng số lượng này không quá lớn, cho nên việc tra cứu và bổ sung là khả thi

Thêm vào đó, cột `Type 1` và `Type 2` không có missing values, chứng tỏ không có Pokemon nào nằm ngoài bộ dữ liệu `pokemon.csv` vô tình lọt vào, do đó cột `Type` cũng có thể được lược bỏ

In [118]:
merged_pokemon = merged_pokemon.drop(columns=['Type'])

<span style = "font-family: Verdana; font-size: 20px">

(Tạm bỏ mấy dòng thiếu để visualize đã, sẽ quay lại fill sau)

In [119]:
merged_pokemon = merged_pokemon.dropna(how='any').reset_index(drop=True)

In [120]:
na = merged_pokemon.isna().sum()
print(f'Missing value in dataset: {sum(na[na > 0])}')

Missing value in dataset: 0


<span style = "font-family: Verdana; font-size: 20px">

##### **3.2. Duplicated Values**
</span>

In [121]:
dup = merged_pokemon.duplicated().sum()
print(f'Duplicated value in dataset: {sum(dup[dup > 0])}')

Duplicated value in dataset: 0


<span style = "font-family: Verdana; font-size: 20px">

##### **3.3. Các vấn đề khác**
</span>

In [122]:
merged_pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 48 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  719 non-null    int64  
 1   Name                719 non-null    object 
 2   Type 1              719 non-null    object 
 3   Type 2              719 non-null    object 
 4   Abilities           719 non-null    object 
 5   HiddenAbility       719 non-null    object 
 6   Generation          719 non-null    object 
 7   Hp                  719 non-null    float64
 8   Attack              719 non-null    float64
 9   Defense             719 non-null    float64
 10  SpecialAttack       719 non-null    float64
 11  SpecialDefense      719 non-null    float64
 12  Speed               719 non-null    float64
 13  TotalStats          719 non-null    float64
 14  Weight              719 non-null    float64
 15  Height              719 non-null    float64
 16  GenderPr

<span style = "font-family: Verdana; font-size: 20px">

Kiểu dữ liệu của ID đang là int, cần chuyển sang str để tránh nhầm lẫn với các cột số liệu khác sau này
</span>

In [123]:
merged_pokemon['ID'] = merged_pokemon['ID'].astype(str)

In [124]:
with open('data/viz_pokemon.csv', 'w', encoding = 'utf-8') as f:
    merged_pokemon.to_csv(f, index=False)